In [1]:


import json 
from netCDF4 import Dataset
import numpy as np
from scipy.spatial.distance import pdist
import random
from sklearn.model_selection import train_test_split
from NeuralNetwork import NeuralNetwork
from Optimizer import Optimizer
import math 
with open('configGA.json') as f:
  config = json.loads(f.read())
import random

print(config)

{'config': {'debug': 'on', 'database': {'file': 'db.dat', 'properties': ['energy', 'gradient'], 'crdmode': 'cartesian', 'timestamp': '2021-04-23T18:25:43.511Z', 'mode': 'ab-initio', 'ab-initio': {'exe': 'QChem.exe', 'chg': 0, 'mult': 1, 'exchange': 'pbe0', 'basis': 'cc-pvdz', 'max_scf_cycles': 500, 'xc_grid': '000075000302', 'mem_static': 4000, 'mem_total': 16000, 'sym_ignore': True, 'set_iter': 50, 'input_bohr': True}}, 'neural_network': {'epochs': 200, 'epoch_step': 50, 'logging_file': 'logfile.txt', 'plotting_file': 'plottingfile.txt', 'optimizer': 'adam', 'activation': 'tanh', 'loss_function': 'MSE', 'model_filename': 'Network.pt'}, 'genetic_algorithm': {'hiddenlayer_size': [10, 100], 'validation_ratio': 0.1, 'hiddenlayer_number': [1, 50], 'learning_rates': [1, 0.0001], 'mutation_rate': 0.1, 'population_size': 6}}}


In [2]:
database = 'db.dat'
data = Dataset(database, mode='r')
if config['config']["database"]["crdmode"] == "cartesian":
    dbset = np.copy(data['crd'])
    coordinatesout = []
    for j,i in enumerate(dbset):
        if j == 0:
            coordinatesout = [pdist(i)]
        else:
            coordinatesout = np.concatenate((coordinatesout,[pdist(i)]))
            
energyout =np.copy(data['energy'])-np.amin(data['energy'])     

coordinates, val_coordinates, output, val_output = train_test_split(coordinatesout, energyout , test_size=config['config']['genetic_algorithm']['validation_ratio'])


In [3]:




# In[ ]:



class GeneticAlgorithm(Optimizer):
    
    
    def __init__(self,config,train_input,train_output,val_input,val_output):
        self.config = config
        self.train_input = train_input
        self.train_output = train_output
        self.val_input = val_input
        self.val_output = val_output
        self.sizebounds = self.config['config']['genetic_algorithm']["hiddenlayer_size"]
        self.layerbounds = self.config['config']['genetic_algorithm']["hiddenlayer_number"]
        self.learningbounds = self.config['config']['genetic_algorithm']["learning_rates"]
        self.mutation_rate = self.config['config']['genetic_algorithm']["mutation_rate"]
        self.population_size = self.config['config']['genetic_algorithm']["population_size"]

    def initializePopulation(self):
        
        sizes = random.sample(range(self.sizebounds[0],self.sizebounds[1]), self.population_size)
        layers = random.sample(range(self.layerbounds[0],self.layerbounds[1]), self.population_size)
        learningrates = np.random.default_rng().uniform(self.learningbounds[0],self.learningbounds[1], self.population_size)
        return(sizes,layers,learningrates)
    
    
    def constructNetworks(self,sizes,layers,learningrates):
        networks = []
        for i in range(self.population_size):
            networks.append(NeuralNetwork(self.config,self.train_input, self.train_output,self.val_input,self.val_output,sizes[i],layers[i],learningrates[i])) 
        return networks
    
    def trainNetworks(self,networks):
        return (list(map(lambda x: x.train(),networks)))
        
    def getParents(self,fitness):
        sort = sorted(fitness)
        print(sort)
        return ((fitness.index(sort[0]), fitness.index(sort[1])),sort[0])
    
    def generatePopulation(self,parents,sizes,layers,learningrates):
        parent1 = (sizes[parents[0]],layers[parents[0]],learningrates[parents[0]])
        parent2 = (sizes[parents[1]],layers[parents[1]],learningrates[parents[1]])

        
            
        
        

    def run(self):
        sizes, layers , learningrates = self.initializePopulation()
        networks = self.constructNetworks(sizes,layers,learningrates)
        fitness = self.trainNetworks(networks)
        parents, lowest_error = self.getParents(fitness)
        best_network = networks[parents[0]].export()
        sizes,layers, learningrates = self.generatePopulation(parents,sizes,layers,learningrates)
        
      
        
        



    

In [4]:




test = GeneticAlgorithm(config,coordinates,  output,val_coordinates, val_output)
test.run()


1.889804564982089 0 81 38 0.9374125440154535
3.3697249081154412 50 81 38 0.9374125440154535
0.26796022752630166 100 81 38 0.9374125440154535
0.031273705620401264 150 81 38 0.9374125440154535
Done Training
2.2100827189585788 0 38 21 0.8450623228342149
1.2234293684090585 50 38 21 0.8450623228342149
0.08949761209624317 100 38 21 0.8450623228342149
0.013003072784817315 150 38 21 0.8450623228342149
Done Training
1.0843809070494133 0 27 35 0.529178813007418
1.062406196695808 50 27 35 0.529178813007418
0.07568074033795875 100 27 35 0.529178813007418
0.008681802561605466 150 27 35 0.529178813007418
Done Training
1.5499259868253699 0 33 18 0.7151737653791662
1.0015153680298765 50 33 18 0.7151737653791662
0.08644041128253356 100 33 18 0.7151737653791662
0.006036473366449093 150 33 18 0.7151737653791662
Done Training
3.4363665708433295 0 82 1 0.8209810648333498
3.896444745986067 50 82 1 0.8209810648333498
0.23503745994325623 100 82 1 0.8209810648333498
0.023348462908392726 150 82 1 0.820981064833